In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np


from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import pickle
from tensorflow import keras

2023-06-09 13:37:27.899412: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 13:37:28.019658: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.3/lib64:/usr/local/cuda-11.3/lib64:
2023-06-09 13:37:28.019695: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-09 13:37:28.048823: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regist

# Data and Model prep

In [2]:
data_directory = "data/HMDA/"

X_test = pd.read_csv(data_directory+'HMDA-MORTGAGE-APPROVAL_Xtest.bz2')
y_test = pd.read_csv(data_directory+'HMDA-MORTGAGE-APPROVAL_ytest.bz2')
X_train = pd.read_csv(data_directory+'HMDA-MORTGAGE-APPROVAL_Xtrain.bz2')
y_train = pd.read_csv(data_directory+'HMDA-MORTGAGE-APPROVAL_ytrain.bz2')

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(650877, 35)
(650877, 1)
(278948, 35)
(278948, 1)


In [3]:
def load_model(model_type):

    if model_type == 'dt':
        model = pickle.load(open(data_directory+'HMDA-MORTGAGE-APPROVAL_DT_scaling_20210205_014819.pkl', 'rb'))
    elif model_type == 'gbc':
        model = pickle.load(open(data_directory+'HMDA-MORTGAGE-APPROVAL_GBC_scaling_20210205_014418.pkl', 'rb'))
    elif model_type == 'lr':
        model = pickle.load(open(data_directory+'HMDA-MORTGAGE-APPROVAL_LR_scaling_20210205_012956.pkl', 'rb'))
    elif model_type == 'rf':
        model =  pickle.load(open(data_directory+'HMDA-MORTGAGE-APPROVAL_RF_scaling_20210205_013239.pkl', 'rb'))
    else:
        model = keras.models.load_model(data_directory+'HMDA-MORTGAGE-APPROVAL_MLP_scaling_20210205_011811.h5')
        
    return model

# Attack

In [4]:
from uret.utils.config import process_config_file

cf = "configs/HMDA/simanneal.yml"
num_samples = 1000
scaler = StandardScaler().fit(X_train)

def feature_extractor(x):
    if len(np.shape(x)) == 2:
        return np.array(scaler.transform(x))
    else:
        return np.array(scaler.transform([x]))

x_transformed = scaler.transform(X_test)

for mt in ['dt', 'gbc', 'lr', 'rf']:
    print("Model type:", mt)
    model = load_model(mt)
    
    model_preds = np.argmax(model.predict_proba(x_transformed),axis=1)
    
    zero_inds = np.where(model_preds == 0)[0][:num_samples]
    one_inds = np.where(model_preds == 1)[0][:num_samples]
    
    task_samples = np.concatenate((list(X_test.values[zero_inds]), list(X_test.values[one_inds])))
    
    orig_model_preds = np.argmax(model.predict_proba(feature_extractor(task_samples)),axis=1)
    
    explorer = process_config_file(cf, model, feature_extractor=feature_extractor, input_processor_list=[])
    adv_samples = explorer.explore(task_samples)  
    
    adv_model_preds = np.argmax(model.predict_proba(feature_extractor(adv_samples)),axis=1)
    
    success_rate = np.sum(adv_model_preds != orig_model_preds)/len(orig_model_preds)
    
    print("Success rate:", success_rate)
    print()
    
    pickle.dump([task_samples, adv_samples], open('data/HMDA_adv_samples/simanneal_adv_samples_' + mt +'.p', 'wb'))

Model type: dt


100%|███████████████████████████████████████| 2000/2000 [34:32<00:00,  1.04s/it]


Success rate: 0.9725

Model type: gbc


100%|███████████████████████████████████████| 2000/2000 [32:53<00:00,  1.01it/s]


Success rate: 0.573

Model type: lr


 46%|██████████████████▌                     | 928/2000 [15:38<17:52,  1.00s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████████| 2000/2000 [49:56<00:00,  1.50s/it]

Success rate: 0.695



In [5]:
cf = "configs/HMDA/simanneal_mlp.yml"
for mt in ['mlp']:
    print("Model type:", mt)
    model = load_model(mt)
    
    model_preds = np.argmax(model.predict(x_transformed),axis=1)
    
    zero_inds = np.where(model_preds == 0)[0][:num_samples]
    one_inds = np.where(model_preds == 1)[0][:num_samples]
    
    task_samples = np.concatenate((list(X_test.values[zero_inds]), list(X_test.values[one_inds])))
    
    orig_model_preds = np.argmax(model.predict(feature_extractor(task_samples)),axis=1)
    
    explorer = process_config_file(cf, model, feature_extractor=feature_extractor, input_processor_list=[])
    adv_samples = explorer.explore(task_samples)  
    
    adv_model_preds = np.argmax(model.predict(feature_extractor(adv_samples)),axis=1)
    
    success_rate = np.sum(adv_model_preds != orig_model_preds)/len(orig_model_preds)
    
    print("Success rate:", success_rate)
    print()
    
    pickle.dump([task_samples, adv_samples], open('data/HMDA_adv_samples/simanneal_adv_samples_' + mt +'.p', 'wb'))

Model type: mlp


2023-06-09 16:08:33.151266: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-09 16:08:33.152178: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.3/lib64:/usr/local/cuda-11.3/lib64:
2023-06-09 16:08:33.152268: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.3/lib64:/usr/local/cuda-11.3/lib64:
2023-06-09 16:08:33.152349: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot

63/63 [==============================] - 0s 871us/step


  0%|                                                  | 0/2000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 16ms/step


...

 27%|██████████▊                             | 540/2000 [17:16<29:00,  1.19s/it]

1/1 [==============================] - 0s 16ms/step


 27%|██████████▊                             | 541/2000 [17:17<28:54,  1.19s/it]

1/1 [==============================] - 0s 16ms/step


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

